In [4]:
import pandas as pd
import numpy as np
import re
from time import time
import preprocess as datapre
import importlib

importlib.reload(datapre)

<module 'preprocess' from 'D:\\GitHub\\MachineLearning\\capstone\\solutions\\preprocess.py'>

In [5]:
train_data = pd.read_csv("../datasets/train.csv")

In [46]:
suffixs = ["AL","AV","BL","CR","CT","DR","EX","HWY","HY","LN","PL","PZ","RD","ST","TR","TER","WY","WAY"]
col_block = "RoadBlock"
col_name1 = "RoadName1"
col_name2 = "RoadName2"
train_data[col_name1] = None
train_data[col_name2] = None
train_data[col_block] = None

# 1200 Block of 3RD ST
block_pattern = r'(^\d{1,}) Block'

# 1200 Block of 3RD ST
name_patterns1 = [r"of (.*?) "+ i + r"$" for i in suffixs]

# 5TH ST / MARKET ST 前半段
name_patterns2 = [r"(.*?) "+ i + r" /" for i in suffixs] 

# 5TH ST / MARKET ST 后半段
name_patterns3 = [r"/ (.*?) " + i + r"$" for i in suffixs]
    

In [47]:
block_map_row = 0
def block_map(a):
    global block_map_row
    block_map_row += 1
    if not isinstance(a, list):
        raise TypeError("block_map, row: {}".format(block_map_row-1))
        
    if len(a) != 0:
        if len(a) > 1:
            print("INFO: block_map row: {} cnt: {} data: {}.".format(block_map_row-1, len(a), a))
        return a[0]
    else:
        return None

blocks = train_data["Address"].str.findall(block_pattern)
train_data[col_block] = blocks.map(block_map)

In [48]:
address_map_row = 0
def address_map(a):
    global address_map_row
    address_map_row += 1
    if not isinstance(a, list):
        raise TypeError("address_map, row: {}".format(address_map_row-1))

    if len(a) != 0:
        if len(a) > 1:
            print("INFO: block_map row: {} cnt: {} data: {}".format(address_map_row-1, len(a), a))
        return a[0]
    else:
        return None

In [51]:
address_map_row=0
address1 = train_data['Address'].str.findall(name_patterns1[0])
for i in name_patterns1[1:]:
    address1 += train_data['Address'].str.findall(i)
address1 = address1.map(address_map)

In [53]:
address1[47:49]

47    None
48    None
Name: Address, dtype: object

In [49]:
address_map_row=0
address2 = train_data["Address"].str.findall(name_patterns2[0])
for i in name_patterns2[1:]:
    address2 += train_data['Address'].str.findall(i)
address2 = address2.map(address_map)

In [50]:
address2[47]

'LINCOLN'

In [ ]:
# address1有地址的行，address2应该没有地址
tmp = address2[address1.isna() == False].isna()
if not tmp.all():
    print("address1 and address2 repeat. index: ", tmp[tmp==False].index)

In [ ]:
address_map_row=0
address3 = train_data["Address"].str.findall(name_patterns3[0])
for i in name_patterns3[1:]:
    address3 += train_data['Address'].str.findall(i)
address3 = address3.map(address_map)

In [ ]:
# address1和address3不应该有重复的地方
tmp = address3[address1.isna() == False].isna()
if not tmp.all():
    print("address1 and address3 repeat. index: ", tmp[tmp==False].index)

In [ ]:
# address2 和 address3 同一行只有一个有地址
tmp = address3[address2.isna()].isna()
if not tmp.all():
    print("address2 and address3 not all None. index: ", tmp[tmp==False].index.shape)

tmp = address2[address3.isna()].isna()
if not tmp.all():
    print("address2 and address3 not all None. index: ", tmp[tmp==False].index.shape)

In [54]:
# merge address1 and address2
tmp = address1.isna()
address1[tmp] = address2[tmp]

In [55]:
address1[47]

'LINCOLN'

In [ ]:
train_data[col_name1] = address1
train_data[col_name2] = address3

In [ ]:
train_data.loc[(train_data[col_block].isna()) & (train_data[col_name1].isna() | train_data[col_name2].isna()),\
               ["Address", col_block, col_name1, col_name2]].index.shape

In [ ]:
train_data[train_data[col_block]=="5900"][["Address", col_block]]

In [6]:
start = time()
a = datapre.extra_address_for_infos(train_data)
print("{} seconds.".format(time()-start))

1.  47      None
48      None
49    HAIGHT
Name: Address, dtype: object
2.  47    LINCOLN
48     SUTTER
49       None
Name: Address, dtype: object
3.  47    LINCOLN
48     SUTTER
49     HAIGHT
Name: Address, dtype: object
There is only one road name in CrossRoad. index:  [173430, 217606, 285516, 350752, 410727, 460380, 526746, 528049, 535469, 673078, 677260, 677268, 683862, 741420, 755445, 756059, 763376, 769674, 769675, 772415, 780968, 787517, 787770, 804816, 804817, 811431, 828322, 828766, 830042, 834617, 834621, 843892, 843893, 870171, 870173, 875737, 877289, 877361]
191.57664108276367 seconds.


In [43]:
a.loc[[47,684,1086]]

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,RoadName1,RoadName2,RoadBlock
47,2015-05-13 20:00:00,LARCENY/THEFT,GRAND THEFT FROM PERSON,Wednesday,TARAVAL,NONE,LINCOLN WY / 14TH AV,-122.471800,37.765687,None,14TH,None
684,2015-05-12 00:15:00,OTHER OFFENSES,TRAFFIC VIOLATION,Tuesday,TARAVAL,"ARREST, BOOKED",LINCOLN WY / 10TH AV,-122.467514,37.765876,None,10TH,None
1086,2015-05-11 02:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,TARAVAL,NONE,LINCOLN WY / 14TH AV,-122.471800,37.765687,None,14TH,None
